# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from tqdm import tqdm
import os
from time import time
from datetime import datetime
import json

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2

# Lots of code in misc
from misc import *

ModuleNotFoundError: No module named 'unityagents'

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [ ]:
env = UnityEnvironment(no_graphics=True, file_name="./Banana_Linux/Banana.x86_64")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while 0:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    print(action)
    if done:                                       # exit loop if episode finished
        break    
    print("Score: {}".format(score))

In [ ]:
# Set some useful parameters, including the seeds used for the test
solved_score = 13
n_episodes = 600
n_rpt = 10
seeds_path = "seeds.np"
device = check_cuda(0)

# We'll need a folder to save all the trained models so we don't have to retrain each time we run the code
if not os.path.isdir("./temp"):
    os.mkdir("./temp")

# We want the same seeds, just in case we run the notebook again and want to re-use old results
if os.path.isfile(seeds_path):
    print("Using saved random seeds from {}".format(seeds_path))
    seeds = np.loadtxt(seeds_path, dtype=np.int32)
else:
    seeds = np.random.randint(0,1e4, size=(n_rpt))
    np.savetxt(seeds_path, seeds, fmt="%d")

In [ ]:
from dqn_agent import Agent

arch_names = ["DQN","DoubleDQN"]
arch_params = {"DQN": {"double": False}, "DoubleDQN": {"double": True}}

all_scores = np.zeros((n_rpt, len(arch_names), n_episodes))

for i, arch in enumerate(arch_names):
    for j, seed in enumerate(seeds):
        t_start = time()
        # First we check if we already trained this agent or not
        save_path = get_model_save_path(arch, n_episodes, seed)
        if not os.path.isfile(save_path):
            print("Architecture {} seed {} started at {}".format(i+1, j+1, datetime.now()), end="")
            # Initialize a new agent with the corresponding random (but presaved) seed
            agent = Agent(device, state_size=state_size, action_size=action_size, 
                                  seed=seed, double=arch_params[arch]["double"])
            
            # Run DQN
            agent, scores, solved_in = dqn(agent, env, solved_score, 
                                                   n_episodes=n_episodes, verbose=False)
            
            # Save model and performance figures
            torch.save(agent.qnetwork_local.state_dict(), save_path)
            np.savetxt(get_scores_save_path(arch, n_episodes, seed), scores, fmt="%f")
            print("... DONE in {:.3f} s".format(time() - t_start))
        else:
            # If we trained the agent, let's load the results
            print("Loading architecture {} seed {} from file".format(i+1, j+1))
            all_scores[j, i] = np.loadtxt(get_scores_save_path(arch, n_episodes, seed), dtype=np.float64)
            
        


In [ ]:
smoothed = smooth_scores(all_scores)

cmap = plt.cm.get_cmap("jet", len(arch_names))
x = np.arange(1, n_episodes+1)
fig, ax = plt.subplots(1)
for i, arch in enumerate(arch_names):
    mu1 = np.mean(smoothed[:,i,:], axis=0)
    sigma1 = np.std(smoothed[:,i,:], axis=0)
    ax.plot(x, mu1, lw=2, label=arch, color=cmap(i))
    ax.fill_between(x, mu1+sigma1, mu1-sigma1, facecolor=cmap(i), alpha=0.5)

ax.set_title("Average score")
ax.legend(loc='upper left')
ax.set_xlabel('episode #')
ax.set_ylabel('Average score');

We can see double DQN performing on par with the vanilla version here. This is not what I expected but it's possible. In the original [double DQN paper](https://arxiv.org/abs/1509.06461) there were a few Atari games in which vanilla DQN actually performed better, for instance Chopper Command, Time Pilot, Robotank, and Assault. 

In [ ]:
env.close()